In [1]:
! sudo pip install snowflake-ml-python==1.0.2 -U 
! pip install snowflake-snowpark-python==1.5.1 -U

     |████████████████████████████████| 1.9MB 7.3MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 81.3MB/s eta 0:00:01
     |████████████████████████████████| 24.6MB 91.3MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 106.2MB/s eta 0:00:01
     |████████████████████████████████| 133kB 123.1MB/s eta 0:00:01
     |████████████████████████████████| 327kB 104.8MB/s eta 0:00:01
     |████████████████████████████████| 12.2MB 103.4MB/s eta 0:00:01
     |████████████████████████████████| 737kB 97.0MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 109.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.3MB/s  eta 0:00:01
     |████████████████████████████████| 200.3MB 28kB/s /s eta 0:00:01
     |████████████████████████████████| 143kB 108.7MB/s eta 0:00:01


     |████████████▉                   | 14.4MB 105.6MB/s eta 0:00:01

     |████████████████████            | 22.5MB 105.6MB/s eta 0:00:01

     |███████████████████████████▍    | 30.8MB 105.6MB/s eta 0:00:01

     |████████████████████████████████| 36.0MB 105.6MB/s eta 0:00:01

     |████████████████████████████████| 36.0MB 105.6MB/s 
     |████████████████████████████████| 307kB 113.4MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 102.7MB/s eta 0:00:01
     |████████████████████████████████| 276kB 106.0MB/s eta 0:00:01
     |████████████████████████████████| 122kB 109.7MB/s eta 0:00:01
     |████████████████████████████████| 225kB 93.8MB/s eta 0:00:01
  Found existing installation: numpy 1.21.1
    Uninstalling numpy-1.21.1:
      Successfully uninstalled numpy-1.21.1
  Found existing installation: pandas 2.0.2
    Uninstalling pandas-2.0.2:
      Successfully uninstalled pandas-2.0.2
  Found existing installation: snowflake-connector-python 2.9.0
    Uninstalling snowflake-connector-python-2.9.0:
      Successfully uninstalled snowflake-connector-python-2.9.0
  Found existing installation: snowflake-snowpark-python 1.0.0
    Uninstalling snowflake-snowpark-python-1.0.0:
      Successfully uninstalled snowflake-snowpark-python-1.0.0
     |████

     |████████████████████████████████| 204kB 89.8MB/s eta 0:00:01
     |████████████████████████████████| 163kB 105.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 2.7MB/s  eta 0:00:01
     |████████████████████████████████| 61kB 2.7MB/s  eta 0:00:01
     |████████████████████████████████| 450kB 104.1MB/s eta 0:00:01
     |████████████████████████████████| 512kB 99.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 6.7MB/s  eta 0:00:01
     |████████████████████████████████| 122kB 104.4MB/s eta 0:00:01
ERROR: snowflake-ml-python 1.0.2 has requirement cryptography<39.0.0,>=3.1.0, but you'll have cryptography 41.0.4 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.23.0 has requirement jsonschema>=4.17.3, but you'll have jsonschema 3.2.0 which is incompatible.


In [2]:
! pip install --upgrade xgboost==1.7.3

     |████████████████████████████████| 193.6MB 22kB/s /s eta 0:00:01
     |████████████████████████████████| 34.5MB 110.0MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 99.3MB/s eta 0:00:01
ERROR: snowflake-ml-python 1.0.2 has requirement cryptography<39.0.0,>=3.1.0, but you'll have cryptography 41.0.4 which is incompatible.


In [1]:
! pip list

Package                    Version     
-------------------------- ------------
absl-py                    1.4.0       
aiohttp                    3.8.5       
aiosignal                  1.3.1       
alembic                    1.11.1      
anyio                      3.7.0       
argon2-cffi                21.3.0      
argon2-cffi-bindings       21.2.0      
asn1crypto                 1.5.1       
asttokens                  2.2.1       
async-generator            1.10        
async-timeout              4.0.3       
attrs                      23.1.0      
Babel                      2.12.1      
backcall                   0.2.0       
beautifulsoup4             4.12.2      
bleach                     4.1.0       
boltons                    23.0.0      
bqplot                     0.12.39     
brotlipy                   0.7.0       
certifi                    2023.7.22   
certipy                    0.1.3       
cffi                       1.15.1      
charset-normalizer         3.2.0       


In [2]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.linear_model import SGDClassifier
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error

In [3]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session



In [4]:
session = get_session()

# Model Building
### Load the final data from snowflake table

In [5]:
table_name = 'predictive_maintenance_final'

In [6]:
sf_df = session.table(table_name).drop('ROW')

In [7]:
sf_df.show(3)

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"           |"LOG_M3"            |"LOG_M4"            |"LOG_M7"  |"LOG_M9"           |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |407438     |1      |3           |S1F0      |1            |0                  |26953834  |0         |0         |2.079441541679836  |0.0                 |3.9702919135521215  |0.0       |2.079441541679836  |
|0          |6          |403174     |1      |3           |S1F0      |1          

In [8]:
a = sf_df.to_pandas()

In [9]:
list(a.columns)

['FAILURE',
 'METRIC5',
 'METRIC6',
 'DAY',
 'DAY_WEEK',
 'SECTOR',
 'OP_PERIOD',
 'DEV_RECONNECTED',
 'MNW1',
 'DIF_M6',
 'DIF_M5',
 'LOG_M2',
 'LOG_M3',
 'LOG_M4',
 'LOG_M7',
 'LOG_M9']

In [10]:
a.head()

,FAILURE,METRIC5,METRIC6,DAY,DAY_WEEK,SECTOR,OP_PERIOD,DEV_RECONNECTED,MNW1,DIF_M6,DIF_M5,LOG_M2,LOG_M3,LOG_M4,LOG_M7,LOG_M9
0,0,6,407438,1,3,S1F0,1,0,26953834,0,0,2.079442,0.000000,3.970292,0.0,2.079442
1,0,6,403174,1,3,S1F0,1,0,7671335,0,0,0.000000,1.386294,0.000000,0.0,0.000000
2,0,12,237394,1,3,S1F0,1,0,21661996,0,0,0.000000,0.000000,0.000000,0.0,0.000000
3,0,6,410186,1,3,S1F0,1,0,9961753,0,0,0.000000,0.000000,0.000000,0.0,0.000000
4,0,15,313173,1,3,S1F0,1,0,16996310,0,0,0.000000,0.000000,0.000000,0.0,1.386294


In [8]:
CATEGORICAL_COLUMNS = ["SECTOR"]
CATEGORICAL_COLUMNS_OE = ["SECTOR_OE"]
NUMERICAL_COLUMNS = ['METRIC5','METRIC6','DAY','DAY_WEEK','OP_PERIOD','MNW1','DIF_M6','DIF_M5','LOG_M2',
 'LOG_M3','LOG_M4','LOG_M7','LOG_M9']
LABEL_COLUMNS = ["FAILURE"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [9]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [10]:
train_df.show(3)

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"FAILURE"  |"METRIC5"  |"METRIC6"  |"DAY"  |"DAY_WEEK"  |"SECTOR"  |"OP_PERIOD"  |"DEV_RECONNECTED"  |"MNW1"    |"DIF_M6"  |"DIF_M5"  |"LOG_M2"  |"LOG_M3"            |"LOG_M4"  |"LOG_M7"  |"LOG_M9"  |
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0          |6          |403174     |1      |3           |S1F0      |1            |0                  |7671335   |0         |0         |0.0       |1.3862943611198906  |0.0       |0.0       |0.0       |
|0          |12         |237394     |1      |3           |S1F0      |1            |0                  |21661996  |0         |0         |0.0       |0.0                 |0.0       |0.0       |0.

Create a pipeline with preprocessing steps and model definition

In [11]:
pipeline = Pipeline(
    steps=[
            (
                "OHE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "classification",
                SGDClassifier(
                input_cols=NUMERICAL_COLUMNS + CATEGORICAL_COLUMNS_OE + ['DEV_RECONNECTED'],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [27]:
model = XGBClassifier(
                input_cols=NUMERICAL_COLUMNS,
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )

In [28]:
model.fit(train_df)
result = model.predict(test_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment
The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [12]:
pipeline.fit(train_df)

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [13]:
result = pipeline.predict(test_df[:20])

The version of package numpy in the local environment is 1.24.4, which does not fit the criteria for the requirement numpy==1.24.3. Your UDF might not work when the package version is different between the server and your local environment


In [14]:
result.show()

SnowparkSQLException: (1304): 01af1ba1-0503-cad2-0072-f3030bf489a6: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555882278/udf_py_2049933684.zip/udf_py_2049933684.py", line 108, in compute
    return lock_function_once(func, invoked)(*[df[idx] for idx in range(df.shape[1])])
  File "/home/udf/32355341555882278/udf_py_2049933684.zip/udf_py_2049933684.py", line 97, in wrapper
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/snowflake/ml/modeling/linear_model/sgd_classifier.py", line 691, in vec_batch_infer
AttributeError: module 'pandas.io.json' has no attribute 'json_normalize'
 in function SNOWML_BATCH_INFERENCE_05A6B717_2FED_439F_AFEC_190615435CD0_PREDICT with handler udf_py_2049933684.compute

In [18]:
pp = pipeline.to_sklearn()

In [24]:
tr = test_df.to_pandas()

In [25]:
result = pp.predict(tr)

/opt/conda/lib/python3.8/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalEncoder was fitted without feature names
  warnings.warn(


IndexError: index 15 is out of bounds for axis 1 with size 15

In [25]:
sf_df1 = session.table("BANK_CUSTOMER_CHURN_PREDICTION").drop('ROW')

In [26]:
CATEGORICAL_COLUMNS = ["COUNTRY", "GENDER"]
CATEGORICAL_COLUMNS_OE = ["COUNTRY_OE", "GENDER_OE"]
NUMERICAL_COLUMNS = ["CREDIT_SCORE", "AGE", "TENURE", "BALANCE", "PRODUCTS_NUMBER", "ESTIMATED_SALARY"]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [27]:
train_df, test_df = sf_df.random_split([0.8,0.2])

In [28]:
pipeline = Pipeline(
    steps=[
            (
                "OE",
                OrdinalEncoder(
                input_cols=CATEGORICAL_COLUMNS,
                output_cols=CATEGORICAL_COLUMNS_OE
                )
            ),
            (
                "MMS",
                MinMaxScaler(
                clip=True,
                input_cols=NUMERICAL_COLUMNS,
                output_cols=NUMERICAL_COLUMNS,
                )
            ),
            (
                "classification",
                XGBClassifier(
                input_cols=CATEGORICAL_COLUMNS_OE+NUMERICAL_COLUMNS + ["CREDIT_CARD"],
                label_cols=LABEL_COLUMNS,
                output_cols=OUTPUT_COLUMNS
                )
            )
    ]
)

In [29]:
pipeline.fit(train_df)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.


In [30]:
result = pipeline.predict(test_df)

The version of package 'numpy' in the local environment is 1.24.4, which does not fit the criteria for the requirement 'numpy==1.24.3'. Your UDF might not work when the package version is different between the server and your local environment.


In [31]:
result.show()

SnowparkSQLException: (1304): 01af1b8a-0503-cace-0072-f3030bf470b2: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/32355341555882206/udf_py_1266416964.zip/udf_py_1266416964.py", line 110, in compute
    return lock_function_once(func, invoked)(*[df[idx] for idx in range(df.shape[1])])
  File "/home/udf/32355341555882206/udf_py_1266416964.zip/udf_py_1266416964.py", line 99, in wrapper
    result = f(*args, **kwargs)
  File "/opt/conda/lib/python3.8/site-packages/snowflake/ml/modeling/xgboost/xgb_classifier.py", line 748, in vec_batch_infer
AttributeError: module 'pandas.io.json' has no attribute 'json_normalize'
 in function SNOWML_BATCH_INFERENCE_4DA7A050_5BDD_404F_ABF7_E6283071B336_PREDICT with handler udf_py_1266416964.compute